Here we'll prepare the data for the GLM analysis.

## Part 0: preparing the first bit of the data

In [1]:
## Spatial stuff
Countries <- c("Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador",
               "Paraguay", "Peru", "Uruguay", "Venezuela")  
library(maptools)
data(wrld_simpl)
world <- wrld_simpl

which.polys <- vector(mode = "list", length(Countries))
for (c in 1:length(Countries)){
  which.polys[[c]] <- grep(paste(Countries[c]), world$NAME)
}
SA <- world[unlist(which.polys), ]

coordsSA <- coordinates(SA)

distmat <- fields::rdist.earth(coordsSA, miles = FALSE)
bordermat <- spdep::nb2mat(spdep::poly2nb(SA), style = "B", zero.policy = TRUE)

Loading required package: sp
Checking rgeos availability: FALSE
 	Note: when rgeos is not available, polygon geometry 	computations in maptools depend on gpclib,
 	which has a restricted licence. It is disabled by default;
 	to enable gpclib, type gpclibPermit()


In [2]:
Trade <- read.csv("../../DATA/TRADE_DATA/all_trade_by_period.csv")

In [3]:
## Exporting data set
covariates <- data.frame(Trade,
                         geodistance = as.vector(distmat),
                         borders = as.vector(bordermat))
write.csv(covariates, "../../DATA/TRADE_DATA/trade_and_geo_covariates.csv", row.names = FALSE)

In [4]:
head(covariates, 3)

from,to,cattleTrade_1986.1995,cattleTrade_1995.2004,cattleTrade_2004.2013,sheepTrade_1986.1995,sheepTrade_1995.2004,sheepTrade_2004.2013,pigsTrade_1986.1995,pigsTrade_1995.2004,pigsTrade_2004.2013,geodistance,borders
Argentina,Argentina,0,0,0,0,0,0,0,0,0,0.000,0
Argentina,Bolivia,71256,64078,960,815,151,0,97,1656,300,2056.028,1
Argentina,Brazil,135072,21423,377,1029,0,226,26,0,0,2970.620,0


### Part 1: process sequence meta data and add more predictors
So we have the trade variables, distance and borders. Now let's chuck the sequence differences in

In [5]:
### Sequence meta data
meta.A <- read.csv("../../DATA/SEQUENCES/data_acquisition/serotype_A_SA_metadata_final.csv")
meta.O <- read.csv("../../DATA/SEQUENCES/data_acquisition/serotype_O_SA_metadata_final.csv")

In [6]:
tabA <- table(meta.A$country)
write.csv ( data.frame(country = names(tabA), samples = as.numeric(tabA) ),
           "../../DATA/XML/glm/serotype_A/predictors_A/samples_A.csv", row.names = FALSE, quote = FALSE)

In [7]:
tabO <- table(meta.O$country)
write.csv ( data.frame(country = names(tabO), samples = as.numeric(tabO) ),
           "../../DATA/XML/glm/serotype_O/predictors_O/samples_O.csv", row.names = FALSE, quote = FALSE)

In [8]:
difference <- function(x) abs(x[1] - x[2])
make_fromTo_df <- function(x, name, FUN = difference){
    inds <- seq_along(x)
    Names <- names(x)
    if(is.null(names(x))) Names <- paste("obs_", inds, sep = "")
    Grid <- expand.grid(inds, inds)
    lst <- lapply(seq_len(nrow(Grid)), function(i) {
        res <- data.frame(
            Names[Grid[i, 1]],
            Names[Grid[i, 2]],
            FUN(c(x[Grid[i, 1]], x[Grid[i, 2]])) 
        )
        names(res) <- c("from", "to", name)
        return(res)
    } )
    final <- do.call(rbind, lst)
    return(final)
}

In [9]:
difference.number.seqs.A <- make_fromTo_df(x = table(meta.A$country), name = "diff_seqs_A", FUN = difference)
product.number.seqs.A <- make_fromTo_df(x = table(meta.A$country), name = "prod_seqs_A", FUN = prod)
seq.info.A <- merge(difference.number.seqs.A, product.number.seqs.A, by = c("from", "to"))
seq.info.A$prod_seqs_A[which(seq.info.A$from == seq.info.A$to)] <- 0

In [11]:
difference.number.seqs.O <- make_fromTo_df(x = table(meta.O$country), name = "diff_seqs_O", FUN = difference)
product.number.seqs.O <- make_fromTo_df(x = table(meta.O$country), name = "prod_seqs_O", FUN = prod)
seq.info.O <- merge(difference.number.seqs.O, product.number.seqs.O, by = c("from", "to"))
seq.info.O$prod_seqs_O[which(seq.info.O$from == seq.info.O$to)] <- 0

In [15]:
all.info.A <- merge(covariates, seq.info.A, by = c("from", "to"), all.x = TRUE)
all.info.O <- merge(covariates, seq.info.O, by = c("from", "to"), all.x = TRUE)

 ### Part 2: Export predictor matrices

In [56]:
Countries <- c("Argentina", "Bolivia", "Brazil",
               "Colombia", "Ecuador",
#                "Paraguay",
               "Peru", "Uruguay", "Venezuela")  

In [57]:
makeMatrix <- function(x, k, name){
  if(length(x) != k^2) stop("data does not appear to be a flattened square matrix")
  m <- matrix(x, ncol = k, nrow = k)
  diag(m) <- 1
  colnames(m) <- rownames(m) <- Countries
  dt <- data.frame(x = Countries, m)
  names(dt) <- c(name, Countries)
  return(dt)
}

In [58]:
forExpo.A <- na.omit( all.info.A[, -c(1:2)] ) + 1
forExpo.A$borders <- forExpo.A$borders - 1

In [60]:
VarNames <-  colnames(forExpo.A)
Dts.A <- lapply(1:ncol(forExpo.A), function(i) makeMatrix(x = forExpo.A[, i],
                                                          k = length(Countries) , name = VarNames[i]))

lapply(1:ncol(forExpo.A), function(i){
      write.csv(Dts.A[[i]],
                file = paste("../../DATA/XML/glm/serotype_A/predictors_A/", VarNames[i], ".csv", sep = ""),
                row.names = FALSE, quote = FALSE)
    }
)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

In [61]:
Countries <- c("Argentina", "Bolivia", "Brazil",
               "Colombia", "Ecuador",
               "Paraguay",
               "Peru", "Uruguay", "Venezuela") 

In [62]:
forExpo.O <- na.omit( all.info.O[, -c(1:2)] ) + 1
forExpo.O$borders <- forExpo.O$borders - 1

In [63]:
VarNames <-  colnames(forExpo.O)
Dts.O <- lapply(1:ncol(forExpo.O), function(i) makeMatrix(x = forExpo.O[, i],
                                                          k = length(Countries) , name = VarNames[i]))

lapply(1:ncol(forExpo.O), function(i){
      write.csv(Dts.O[[i]],
                file = paste("../../DATA/XML/glm/serotype_O/predictors_O/", VarNames[i], ".csv", sep = ""),
                row.names = FALSE, quote = FALSE)
    }
)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL